# Détection des sources sismiques

In [ ]:
import pandas as pd
import numpy as np
import struct
import matplotlib
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join

from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.utilities.dataframe_functions import impute

from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import classification_report

%matplotlib inline

In [ ]:
import xgboost as xgb

In [ ]:
matplotlib.rcParams['figure.figsize'] = (20, 10)

## Chargement des données

In [47]:
path = "SUHA_Paris2017"
part = 0 # choisir parmi 0, 1, 2, 3, 4
nb_part = 5

all_files = [path + "/" + f for f in listdir(path) if isfile(join(path, f))]

start = part * int(len(all_files) / nb_part)
stop = (part + 1) * int(len(all_files) / nb_part)
some_files = all_files[start:stop]

In [49]:
n1_convH = 75
n2_convH = 5400
n3_convH = 100

In [62]:
def load_data(filename):
    """
    Charger le fichier de données
    """
    n_elems = n1_convH * n2_convH * n3_convH
    f = open(filename, "rb")
    data = struct.unpack('f' * n_elems, f.read(4 * n_elems))
    f.close()
    return np.reshape(data, (n3_convH, n2_convH, n1_convH))

In [63]:
convH = load_data(all_files[50])

## Normalisation

In [59]:
def normalize_signals(data):
    """
    Normaliser le signal par rapport au temps
    """
    for receiver in range(n3_convH):
        for trace in range(n2_convH):
            data[receiver, trace] = data[receiver, trace, :] / np.amax(data[receiver, trace], axis=0)
    return data

In [60]:
n_convH = normalize_signals(convH)

## Représentation des traces sismiques après convolution

In [ ]:
receiver = 50

In [ ]:
plt.close('all')
plt.imshow(n_convH[receiver].T, cmap='gray', aspect="auto")
plt.xlabel("Trace sismique après convolution")
plt.ylabel("Temps (en 20 ms)")
plt.title("Trace sismique après convolution pour un récepteur")
plt.show()

In [ ]:
def compute_sums(data, n_tracesByGroups=20):
    """
    Calculer la somme des traces par groupe de 20
    """
    n3, n2, n1 = data.shape
    n_groups = int(n2 / n_tracesByGroups)

    sum_traces_grouped = np.zeros((n3, n_groups, n1))
    sum_traces = np.zeros((n3, n1))
    
    for receiver in range(n3):
        traces = data[receiver]
        sum_traces[receiver, :] = traces.sum(axis=0) / np.amax(np.abs(traces.sum(axis=0)))
        for n_group in range(n_groups):
            start = n_tracesByGroups * n_group
            stop = n_tracesByGroups * (n_group + 1)
            sum_traces_grouped[receiver, n_group, :] = traces[start:stop, :].sum(axis=0) / np.amax(np.abs(traces[start:stop, :].sum(axis=0)))
    
    return sum_traces, sum_traces_grouped

In [ ]:
def compute_SSE(u_vgt, u):
    """
    Calculer la somme de l'écart quadratique
    """
    n3, n2, n1 = u.shape
    SSE = np.zeros((n3, n2))
    for receiver in range(n3):
        for group in range(n2):
            SSE[receiver, group] = np.square(u_vgt[receiver] - u[receiver, group, :]).sum()
    return SSE

In [ ]:
sum_traces, sum_traces_grouped = compute_sums(n_convH, n_tracesByGroups=20)

In [ ]:
plt.close('all')
plt.imshow(sum_traces.T, cmap='gray', aspect="auto")
plt.xlabel("Récepteurs")
plt.ylabel("Temps (en 20 ms)")
plt.title("Somme des traces sismiques après convolution")
plt.show()

In [ ]:
sumH = compute_SSE(sum_traces, sum_traces_grouped)

In [ ]:
plt.plot(sumH[receiver], 'o')
plt.show()

## Etiquettage des traces sismiques groupées

- Classe 0: la trace sismique groupée ressemble à la sommation des traces sismiques sur 6 heures
- Classe 1: la trace sismique groupée ne ressemble pas à la sommation des traces sismiques sur 6 heures

In [ ]:
receiver = 51

In [ ]:
def labelize(sumH, receiver):
    """
    Etiquetter l'échantillon
    """
    decision = np.median(sumH[receiver]) # 2.5
    y_todf = np.zeros(sumH[receiver].shape)
    y_todf = y_todf + (sumH[receiver] < decision).astype(np.int)
    return pd.DataFrame(y_todf)[0]

In [ ]:
y = labelize(sumH, receiver)

In [ ]:
plt.plot(y, 'o')
plt.show()

## Extraction des caractéristiques

In [ ]:
def extractFeatures(sum_traces_grouped, receiver):
    """
    Extraire les caractéristiques pour chaque récepteur
    """
    master_df = pd.DataFrame(sum_traces_grouped[receiver, 0])
    master_df['id'] = 0
    for ii in range(1, sum_traces_grouped.shape[1]):
        temp_df = pd.DataFrame(sum_traces_grouped[receiver, ii])
        temp_df['id'] = ii
        master_df = pd.DataFrame(np.vstack([master_df, temp_df]))
    # 75 * 270 -1 = 20 249
    
    extraction_settings = ComprehensiveFCParameters()
    extraction_settings.IMPUTE = impute # Interpolation pour éviter les valeurs NaN
    return extract_features(master_df, column_id=1, default_fc_parameters=extraction_settings).sort_index()

In [ ]:
%time X = extractFeatures(sum_traces_grouped, receiver)

## Premiers modèles

In [ ]:
def classification_metrics(y, y_pred):
    """
    Métriques de classification
    """
    print(classification_report(y, y_pred))

#### Méthodes d'ensemble

In [ ]:
clf = DecisionTreeClassifier()
%time y_pred = cross_val_predict(clf, X, y, cv=10)
classification_metrics(y, y_pred)

In [ ]:
tree = DecisionTreeClassifier()
clf = BaggingClassifier(tree, n_estimators=200)
%time y_pred = cross_val_predict(clf, X, y, cv=10)
classification_metrics(y, y_pred)

In [ ]:
clf = RandomForestClassifier(n_estimators=200)
%time y_pred = cross_val_predict(clf, X, y, cv=10)
classification_metrics(y, y_pred)

In [ ]:
clf = ExtraTreesClassifier(n_estimators=200)
%time y_pred = cross_val_predict(clf, X, y, cv=10)
classification_metrics(y, y_pred)

In [ ]:
def xgb_cross_val_predict(params, X, y, cv=10):
    """
    Cross validation avec Xgboost
    """
    dX = xgb.DMatrix(X.values, label=y.values)

    n, p = X.shape
    print(n, p)
    
    bst = xgb.train(params, dX, num_round)

    preds = bst.predict(dX)
    return preds

In [ ]:
params = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic'}
num_round = 2
%time y_pred = xgb_cross_val_predict(params, X, y, cv=10)
# classification_metrics(y, y_pred)

#### Réseaux de neurones

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(300, 50, 100),
                  activation='relu', solver='adam', max_iter=500, early_stopping=False)
%time y_pred = cross_val_predict(clf, X, y, cv=10)
classification_metrics(y, y_pred)

#### Machine à vecteur support

In [ ]:
clf = SVC(C=0.5, kernel='rbf', degree=3)
%time y_pred = cross_val_predict(clf, X, y, cv=10)
classification_metrics(y, y_pred)

https://xgboost.readthedocs.io/en/latest/

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html

## Importance des caractéristiques

In [ ]:
def explicability(weights, columns, first=10):
    """
    Représenter le poids des variables explicatives
    """
    weights = weights.reshape(-1)
    idx = weights.argsort()[::-1][:first]
    x = np.arange(first)
    y = weights[idx]
    labels = columns[idx]

    plt.close('all')
    fig, ax = plt.subplots(figsize=(18, 4))

    ax.bar(x, y, width=0.5, align='center')
    ax.set_xlabel(u"Variables explicatives")
    ax.set_ylabel(u"Poids")

    plt.xticks(x, labels, rotation=80)
    plt.title(u"Poids des variables explicatives")
    plt.show()
    
    filename = "features_weights.csv"
    features_weights = []
    with open(filename, 'w') as f:
        for x in zip(labels, y):
            f.write(x[0])
            f.write("\t")
            f.write(str(x[1]))
            f.write("\n")
            features_weights.append(x)
    return features_weights

In [ ]:
def score_with_components(clf, X, y, columns, weights, cv=10):
    """
    Calculer les scores en fonction du nombre de composants
    """
    dict_feat_import = {
        'feature_names': list(columns),
        'feat_importance': list(weights)
    }

    feat_import_df = pd.DataFrame.from_dict(dict_feat_import, orient='columns')
    sort_feat_import_df = feat_import_df.sort_values(by=['feat_importance'],ascending=[0]).reset_index(drop=True)
    sorted_vect_importance = np.sort(weights, kind='heapsort')

    sorted_vect_importance[:] = sorted_vect_importance[::-1]
    score_mean = []
    error_on_mean_score = []
    n_th_most_important_features = []
    last_included_feat = []
    df_X = pd.DataFrame(X, columns=columns)
    df_y = pd.DataFrame(y)

    n_split_cv_n_fold = cv

    for ind in range(sort_feat_import_df.shape[0]):
        if ind % 10 == 0:
            print('ind = ' + ind.__str__())

        # print("Variables explicatives:")
        # print(list(sort_feat_import_df['feature_names'].head(ind+1)))
        last_included_feat.append(list(sort_feat_import_df['feature_names'].head(ind+1))[-1])
        X_new = df_X[list(sort_feat_import_df['feature_names'].head(ind+1))]

        # print(X_new.shape)
        # print("X_new.shape")
        # print('cross val...')

        scores = cross_val_score(clf, X_new, np.ravel(df_y), cv=n_split_cv_n_fold, n_jobs=-1)
        # print(np.std(scores))
        error_on_mean_score.append(np.std(scores) / np.sqrt(n_split_cv_n_fold - 1))
        # print("std scores")
        # print(np.mean(scores))
        score_mean.append(np.mean(scores))
        n_th_most_important_features.append(ind+1)

        # print("mean scores")

        # print('************* \n last included feature and score')
        """for a in range(len(last_included_feat)):
            print(last_included_feat[a] + u"\u0009"*4+'  -->  mean score  :  ' + score_mean[a].__str__())"""
        # print('************* \n last included feature and score (above)')

        # print('******************** \n')
    return n_th_most_important_features, score_mean, error_on_mean_score, sort_feat_import_df

In [ ]:
def plot_score_with_components(n_th_most_important_features, score_mean,
                               error_on_mean_score, sort_feat_import_df):
    """
    Tracer les scores en fonction du nombre de composants
    """
    x = n_th_most_important_features
    y = score_mean
    yerr = error_on_mean_score
    plt.clf()
    plt.close()
    plt.close('all')

    fig = plt.figure(figsize=(22.0, 13.0))
    plt.errorbar(x, y, yerr=yerr, fmt='o', linestyle='-', color='b')
    plt.title('mean score for n-th most important features', fontsize=22)
    ax = plt.gca()
    ax.legend_ = None
    plt.xlabel('nb of components considered ', fontsize=22)
    plt.ylabel('scores',fontsize=22)
    plt.legend(numpoints=1, loc=2)  # numpoints = 1 for nicer display
    axes = plt.gca()
    axes.set_xlim([0, sort_feat_import_df.shape[0]])
    plt.tight_layout()
    plt.show()

    plt.clf()
    plt.close()
    plt.close('all')

In [ ]:
clf = ExtraTreesClassifier(n_estimators=200)
clf.fit(X, y)
weights = clf.feature_importances_
columns = X.columns

In [ ]:
explicability(weights, columns, first=25)

In [ ]:
most_important_features, score_mean, error_on_mean_score, sort_feat_import_df = score_with_components(
    clf, X, y, columns, weights, cv=10)

In [ ]:
plot_score_with_components(most_important_features, score_mean, error_on_mean_score, sort_feat_import_df)

## Entrainement sur un récepteur et prédiction sur un autre

In [72]:
receiver = 50

y = labelize(sumH, receiver)
X = extractFeatures(sum_traces_grouped, receiver)

clf = ExtraTreesClassifier(n_estimators=200)
clf.fit(X, y)

classification_metrics(y, clf.predict(X))

Feature Extraction: 100%|██████████| 270/270 [00:25<00:00, 10.39it/s]


             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00       135
        1.0       1.00      1.00      1.00       135

avg / total       1.00      1.00      1.00       270



In [74]:
receiver = 51

y = labelize(sumH, receiver)
X = extractFeatures(sum_traces_grouped, receiver)

classification_metrics(y, clf.predict(X))

Feature Extraction: 100%|██████████| 270/270 [00:24<00:00, 11.04it/s]


             precision    recall  f1-score   support

        0.0       0.50      1.00      0.67       135
        1.0       0.00      0.00      0.00       135

avg / total       0.25      0.50      0.33       270



/opt/miniconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Prédiction pour plusieurs récepteurs

In [67]:
def fitPredictAllReceivers(receivers, sumH, sum_traces_grouped, echantillonage=2, debug=True):
    """
    Entraîner et prédire un classifieur pour chaque récepteur
    """
    for receiver in receivers:
        if receiver % echantillonage == 0:
            print("Recepteur %d" %(receiver))
            y = labelize(sumH, receiver)
            X = extractFeatures(sum_traces_grouped, receiver)

            clf = ExtraTreesClassifier(n_estimators=200)
            y_pred = cross_val_predict(clf, X, y, cv=10)
            
            if debug:
                classification_metrics(y, y_pred)

In [ ]:
%time fitPredictAllReceivers(range(n3_convH), sumH, sum_traces_grouped, echantillonage=2, debug=True)

## Pédiction pour plusieurs fichiers et récepteurs

In [75]:
def fitPredictAllFiles(files, echantillonage=2, debug=True):
    """
    Entraîner et prédire un classifieur pour chaque récepteur des fichiers
    """
    for file in files:
        print("Fichier %d" %(file))
        convH = load_data(file)
        n_convH = normalize_signals(convH)
        sum_traces, sum_traces_grouped = compute_sums(n_convH, n_tracesByGroups=20)
        sumH = compute_SSE(sum_traces, sum_traces_grouped)
        
        fitPredictAllReceivers(range(n3_convH), sumH, sum_traces_grouped, echantillonage, debug=debug)

In [76]:
fitPredictAllFiles(some_files[:2], echantillonage=5, debug=True)

Recepteur 0


Feature Extraction: 100%|██████████| 270/270 [00:24<00:00, 11.06it/s]


             precision    recall  f1-score   support

        0.0       0.84      0.83      0.84       135
        1.0       0.83      0.84      0.84       135

avg / total       0.84      0.84      0.84       270

Recepteur 5


Feature Extraction: 100%|██████████| 270/270 [00:24<00:00, 10.94it/s]


             precision    recall  f1-score   support

        0.0       0.86      0.89      0.87       135
        1.0       0.88      0.85      0.87       135

avg / total       0.87      0.87      0.87       270

Recepteur 10


...

Feature Extraction: 100%|██████████| 270/270 [00:25<00:00, 10.74it/s]


             precision    recall  f1-score   support

        0.0       0.87      0.86      0.86       135
        1.0       0.86      0.87      0.86       135

avg / total       0.86      0.86      0.86       270

Recepteur 20


Feature Extraction:  83%|████████▎ | 224/270 [00:21<00:04, 10.26it/s]Process ForkPoolWorker-305:
KeyboardInterrupt
Process ForkPoolWorker-302:
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process ForkPoolWorker-304:
  File "/opt/miniconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/miniconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
Process ForkPoolWorker-303:
  File "/opt/miniconda3/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/opt/miniconda3/lib/python3.6/site-packages/tsfresh/feature_extraction/extraction.py", line 504, in _extract_features_for_one_time_series
    current_result = dataframe.groupby(column_id)[column_value].apply(apply_function, **kwargs).unstack()
  File 

KeyboardInterrupt: 

  File "/opt/miniconda3/lib/python3.6/site-packages/pandas/core/series.py", line 1475, in autocorr
    return self.corr(self.shift(lag))

  File "/opt/miniconda3/lib/python3.6/site-packages/pandas/core/groupby.py", line 2656, in aggregate
    (_level or 0) + 1)
  File "/opt/miniconda3/lib/python3.6/site-packages/pandas/core/series.py", line 1422, in corr
    min_periods=min_periods)
  File "/opt/miniconda3/lib/python3.6/site-packages/pandas/core/generic.py", line 4483, in _align_series
    if self.index.equals(other.index):
  File "/opt/miniconda3/lib/python3.6/site-packages/pandas/core/groupby.py", line 2718, in _aggregate_multiple_funcs
    results[name] = obj.aggregate(func)
  File "/opt/miniconda3/lib/python3.6/site-packages/pandas/core/nanops.py", line 50, in _f
    return f(*args, **kwargs)
KeyboardInterrupt
  File "/opt/miniconda3/lib/python3.6/site-packages/pandas/core/groupby.py", line 2666, in aggregate
    return self._python_agg_general(func_or_funcs, *args, **kwargs)
  Fil